# Perfume

# Proje Açıklaması
Bu proje, parfüm verilerini analiz etmek ve en popüler parfümleri, notaları ve markaları incelemek amacıyla gerçekleştirilmiştir. Veriler, Fragrantica'dan alınan parfüm bilgilerini içermektedir. Projede, parfüm notalarının dağılımı, en popüler notalar, en olumlu ve olumsuz yorumlara sahip parfümler ile marka analizleri yapılmıştır. Ayrıca, parfüm öneri sistemi de geliştirilmiştir.


## Kullanılan Veriler
Veriler, [Fragrantica Data | Kaggle](https://www.kaggle.com/datasets/joehusseinmama/fragrantica-data/data) adresinden temin edilmiştir. Bu veri seti, parfüm başlıkları, markaları, notaları ve kullanıcı puanlarını içermektedir.

In [2]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from textblob import TextBlob
import pandas as pd
import ast
from itertools import combinations
from collections import Counter

### Veriyi yükle

In [4]:
df2 = pd.read_csv("perfumes_table.csv")

In [5]:
df2.shape

(84144, 7)

In [6]:
df2.sample(5)

,rating,notes,designer,reviews,description,url,title
44968,NaN,"['Chinotto', 'Bitter Orange', 'Juniper', 'Card...",tuttotondo perfumes and colognes,['Fresh aromatic citrus with juniper that smel...,Chinotto by Tuttotondo is a Amber Floral fragr...,https://www.fragrantica.com/perfume/Tuttotondo...,Chinotto Tuttotondo for women and men
50596,NaN,"['Fir', 'Pine', 'Siberian Pine', 'Juniper', 'M...",wood moss perfumes and colognes,[],Ernak by Wood Moss is a Woody fragrance for wo...,https://www.fragrantica.com/perfume/Wood-Moss/...,Ernak Wood Moss for women and men
3961,NaN,"['Lily', 'Floral Notes', 'Honey', 'Moss', 'Gre...",black phoenix alchemy lab perfumes and colognes,[],The Phantom Wooer by Black Phoenix Alchemy Lab...,https://www.fragrantica.com/perfume/Black-Phoe...,The Phantom Wooer Black Phoenix Alchemy Lab fo...
59596,NaN,"['Bergamot', 'Orange', 'Ylang-Ylang', 'Bulgari...",haas parfum perfumes and colognes,[],Haas No.5 La femme de Candide by Haas Parfum i...,https://www.fragrantica.com/perfume/Haas-Parfu...,Haas No.5 La femme de Candide Haas Parfum for ...
21838,NaN,"['Lavender', 'Green Notes', 'Bergamot', 'Nerol...",giorgio beverly hills perfumes and colognes,"[""I first bought Wings back in 2004 and it gar...",Wings for Men by Giorgio Beverly Hills is a Ar...,https://www.fragrantica.com/perfume/Giorgio-Be...,Wings for Men Giorgio Beverly Hills for men


In [7]:
df2.isnull().sum()

rating         81670
notes              0
designer           0
reviews            0
description        8
url                0
title              8
dtype: int64

In [8]:
84144-81670

2474

In [9]:
df2 = df2.dropna(subset=['rating'])

In [10]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2474 entries, 0 to 3214
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   rating       2474 non-null   float64
 1   notes        2474 non-null   object 
 2   designer     2474 non-null   object 
 3   reviews      2474 non-null   object 
 4   description  2466 non-null   object 
 5   url          2474 non-null   object 
 6   title        2466 non-null   object 
dtypes: float64(1), object(6)
memory usage: 154.6+ KB


In [11]:
df2['rating'].mean()

3.9900606305578012

In [12]:
df2['rating'].describe()

count    2474.000000
mean        3.990061
std         0.512335
min         1.000000
25%         3.790000
50%         4.000000
75%         4.250000
max         5.000000
Name: rating, dtype: float64

In [13]:
pd.set_option('display.max_rows', None)

In [14]:
df2['notes'].unique()

array(["['Vanila', 'Madagascar Vanilla', 'Vanilla Absolute']",
       "['Mint', 'Lavender', 'Nutmeg', 'elemi', 'Cypress', 'Sandalwood', 'Saffron', 'Cypriol Oil or Nagarmotha', 'Amber', 'Patchouli', 'Precious Woods', 'Musk']",
       "['Neroli', 'Vetiver', 'Basil', 'Fig']", ...,
       "['Blood Orange', 'Grapefruit', 'Black Pepper', 'cannabis', 'Carnation', 'Ginger', 'Olibanum', 'Agarwood (Oud)', 'Styrax', 'Tobacco', 'Vetiver']",
       "['Ambrette (Musk Mallow)', 'Sea Salt', 'Mineral notes', 'Sage']",
       "['Pear', 'Bergamot', 'Black Currant', 'Mandarin Orange', 'Gardenia', 'Cyclamen', 'Rose', 'Musk', 'Sandalwood', 'Iris']"],
      dtype=object)

### En sık kullanılan notalar

In [16]:
notes_list = df2['notes'].apply(lambda x: x.split(',')).explode()
top_notes = notes_list.value_counts().head(10)

In [17]:
df2.sample(5)

,rating,notes,designer,reviews,description,url,title
2137,2.50,"['Bergamot', 'Lemon', 'Galbanum', 'Violet', 'J...",yves de sistelle perfumes and colognes,['Old Spice lime edition'],Aviator Style by Yves de Sistelle is a Woody f...,https://www.fragrantica.com/perfume/Yves-de-Si...,Aviator Style Yves de Sistelle for men
799,3.82,"['Raspberry', 'Orange', 'Bergamot', 'Orange Bl...",alexandre.j perfumes and colognes,"[""This is just nice. Very feminine and frilly,...",Oscent Pink by Alexandre.J is a Floral Fruity ...,https://www.fragrantica.com/perfume/Alexandre-...,Oscent Pink Alexandre.J for women
2421,5.00,"['Camphor', 'Mint', 'Woody Notes', 'Dried Frui...",maison anthony marmin perfumes and colognes,['Divine!'],Sheikh Al Arabi by Maison Anthony Marmin is a ...,https://www.fragrantica.com/perfume/Maison-Ant...,Sheikh Al Arabi Maison Anthony Marmin for wome...
2067,5.00,"['Cumin', 'Mandarin Orange', 'Jasmine Sambac',...",maison douze perfumes and colognes,[],A Mort Scorpio by Maison Douze is a fragrance ...,https://www.fragrantica.com/perfume/Maison-Dou...,A Mort Scorpio Maison Douze for women and men
614,4.18,"['Resins', 'Vanilla', 'Oriental notes', 'Citru...",dsh perfumes perfumes and colognes,['Dessert Shalimar. The bergamot is toned down...,Egyptian Shalimar by DSH Perfumes is a Amber f...,https://www.fragrantica.com/perfume/DSH-Perfum...,Egyptian Shalimar DSH Perfumes for women and men


### En Popüler Tasarımcılar

In [19]:
designer_counts = df2['designer'].value_counts().head(10)

### Öncelikle parfümleri unisex, erkek ve kadın olarak sınıflandıralım

In [21]:
def categorize_fragrance(title):
    if pd.isna(title):
        return "Bilinmiyor"
    title_lower = title.lower()
    if "for women and men" in title_lower or "for men and women" in title_lower:
        return "Unisex"
    elif "for women" in title_lower:
        return "Kadın"
    elif "for men" in title_lower:
        return "Erkek"
    else:
        return "Bilinmiyor"

In [22]:
df2['category'] = df2['title'].apply(categorize_fragrance)

In [23]:
category_counts = df2['category'].value_counts()

### En yüksek puanlı parfümleri bulalım
### En iyi 10 parfüm

In [25]:
top_fragrances = df2.sort_values('rating', ascending=False).head(10)
for idx, row in top_fragrances.iterrows():
    print(f"{row['title']} - Rating: {row['rating']} - Marka: {row['designer']}")

Vanilla Scent Fiorucci for women and men - Rating: 5.0 - Marka: fiorucci perfumes and colognes
Satori Fusho Organic Oud Ensar Oud for women and men - Rating: 5.0 - Marka: ensar oud perfumes and colognes
Eau de Parfum Baies Exquises Durance en Provence for women and men - Rating: 5.0 - Marka: durance en provence perfumes and colognes
Amuro 7 Dzintars for women - Rating: 5.0 - Marka: dzintars perfumes and colognes
Горная фиалка (Mountain violet) Flora for women - Rating: 5.0 - Marka: flora perfumes and colognes
Sky Blue Novae Plus for women - Rating: 5.0 - Marka: novae plus perfumes and colognes
Orris TEAsense 鸢尾岩茶 Mobius Fragrances 莫比乌斯 for women and men - Rating: 5.0 - Marka: mobius fragrances 莫比乌斯 perfumes and colognes
Legend FA'RA for men - Rating: 5.0 - Marka: fa'ra perfumes and colognes
Queen In The War Prince War for women - Rating: 5.0 - Marka: prince war perfumes and colognes
Un Baiser sur le Pont Neuf Condensé Paris for women and men - Rating: 5.0 - Marka: condensé paris perfum

### Parfüm Notalarını Çıkarma ve Analiz Etme

In [27]:
import ast
from collections import Counter

# String olarak saklanan notaları gerçek liste haline dönüştüren fonksiyon
def extract_notes(notes_str):
    if pd.isna(notes_str):
        return []
    try:
        if isinstance(notes_str, str) and (notes_str.startswith('[') or notes_str.startswith('[')):
            return ast.literal_eval(notes_str)
        else:
            return [notes_str]  # Eğer liste değilse, tek elemanlı liste olarak döndür
    except:
        return []

In [28]:
# Notları çıkaralım
df2['notes_list'] = df2['notes'].apply(extract_notes)

# Tüm notları düzleştir ve en popüler notları bul
all_notes = []
for notes in df2['notes_list']:
    all_notes.extend(notes)

# En popüler 20 notayı bul
most_common_notes = Counter(all_notes).most_common(20)
print("\nEn Popüler 20 Parfüm Notu:")
for note, count in most_common_notes:
    print(f"{note}: {count} kez kullanılmış")


En Popüler 20 Parfüm Notu:
Musk: 942 kez kullanılmış
Amber: 694 kez kullanılmış
Jasmine: 660 kez kullanılmış
Bergamot: 657 kez kullanılmış
Sandalwood: 646 kez kullanılmış
Vanilla: 595 kez kullanılmış
Patchouli: 559 kez kullanılmış
Rose: 559 kez kullanılmış
Cedar: 446 kez kullanılmış
Vetiver: 313 kez kullanılmış
Mandarin Orange: 297 kez kullanılmış
Tonka Bean: 259 kez kullanılmış
Lavender: 251 kez kullanılmış
Lemon: 239 kez kullanılmış
Orange Blossom: 211 kez kullanılmış
Lily-of-the-Valley: 201 kez kullanılmış
Cardamom: 190 kez kullanılmış
Pink Pepper: 190 kez kullanılmış
Violet: 187 kez kullanılmış
Iris: 176 kez kullanılmış


### Kategorilere göre ortalama puanları hesaplayalım

In [30]:
avg_rating_by_category = df2.groupby('category')['rating'].mean()

### Yorumların duygu analizi

In [32]:
from textblob import TextBlob
import numpy as np

# Yorumları çıkaran fonksiyon
def extract_reviews(reviews_str):
    if pd.isna(reviews_str):
        return []
    try:
        if isinstance(reviews_str, str) and (reviews_str.startswith('[') or reviews_str.startswith('[')):
            return ast.literal_eval(reviews_str)
        else:
            return [reviews_str]
    except:
        return []

# Yorumları çıkaralım
df2['reviews_list'] = df2['reviews'].apply(extract_reviews)

In [33]:
# Her parfüm için duygu skoru hesaplayan fonksiyon
def calculate_sentiment(reviews):
    if not reviews:
        return np.nan
    
    total_sentiment = 0
    for review in reviews:
        if review and isinstance(review, str):
            blob = TextBlob(review)
            total_sentiment += blob.sentiment.polarity
    
    return total_sentiment / len(reviews) if reviews else np.nan

# Duygu skorlarını hesaplayalım
df2['sentiment_score'] = df2['reviews_list'].apply(calculate_sentiment)

# Duygu skorları ve puanlar arasındaki ilişkiyi inceleyelim
correlation = df2['sentiment_score'].corr(df2['rating'])
print("\nDuygu Skoru ve Puan Arasındaki Korelasyon:")
print(f"Korelasyon: {correlation:.4f}")


Duygu Skoru ve Puan Arasındaki Korelasyon:
Korelasyon: 0.1555


### En olumlu yorumlara sahip 5 parfümü bul

In [35]:
top_positive = df2.sort_values('sentiment_score', ascending=False).head(5)
print("\nEn Olumlu Yorumlara Sahip 5 Parfüm:")
for idx, row in top_positive.iterrows():
    print(f"{row['title']} - Sentiment: {row['sentiment_score']:.4f} - Rating: {row['rating']}")


En Olumlu Yorumlara Sahip 5 Parfüm:
Kyoto by Night Proad for women and men - Sentiment: 1.0000 - Rating: 4.5
Cardamom & Vanilla Blue Scents for men - Sentiment: 1.0000 - Rating: 4.75
Bogner Woman No.1 Summer Edition Bogner for women - Sentiment: 0.7800 - Rating: 4.2
Spot Sunset for Him Hinode for men - Sentiment: 0.7500 - Rating: 3.67
From Zara With Love Summer Edition Zara for women - Sentiment: 0.7201 - Rating: 3.91


### En olumsuz yorumlara sahip 5 parfümü bul

In [37]:
top_negative = df2.sort_values('sentiment_score').head(5)
print("\nEn Olumsuz Yorumlara Sahip 5 Parfüm:")
for idx, row in top_negative.iterrows():
    print(f"{row['title']} - Sentiment: {row['sentiment_score']:.4f} - Rating: {row['rating']}")


En Olumsuz Yorumlara Sahip 5 Parfüm:
Muskara Vetiveria Fueguia 1833 for women and men - Sentiment: -0.8750 - Rating: 3.7
No. 4 FL Parfums for men - Sentiment: -0.8500 - Rating: 3.5
Caliente Perfume A Wing & A Prayer Perfumes for women and men - Sentiment: -0.7000 - Rating: 4.0
800 Black Zara for men - Sentiment: -0.5250 - Rating: 4.0
Acqua Di Parisis Roma Reyane Tradition for men - Sentiment: -0.5107 - Rating: 3.0


### Öneri sistemi oluşturalım

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Parfüm notlarını birleştiren fonksiyon
def create_notes_string(notes_list):
    if not notes_list:
        return ""
    return " ".join(notes_list)

# Notları birleştir
df2['notes_string'] = df2['notes_list'].apply(create_notes_string)

# Açıklamaları ve notları birleştirerek özellik vektörü oluştur
df2['features'] = df2['notes_string'] + " " + df2['description'].fillna("")

# TF-IDF vektörizasyon uygulayalım
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df2['features'])

# Kosinüs benzerlik matrisi oluştur
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [40]:
# Bir parfüm için benzer parfümleri öneren fonksiyon
def get_recommendations(title, cosine_sim=cosine_sim):
    try:
        # Parfümün indeksini bul
        idx = df2[df2['title'] == title].index[0]
        
        # Benzerlik skorlarını al
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Benzerlik skorlarına göre sıralayalım
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # En benzer 5 parfümü alalım (kendisi hariç)
        sim_scores = sim_scores[1:6]
        
        # Parfüm indekslerini al
        fragrance_indices = [i[0] for i in sim_scores]
        
        # Sonuçları döndürelim
        result = pd.DataFrame({
            'title': df2['title'].iloc[fragrance_indices].values,
            'designer': df2['designer'].iloc[fragrance_indices].values,
            'similarity': [i[1] for i in sim_scores],
            'rating': df2['rating'].iloc[fragrance_indices].values
        })
        
        return result
    except:
        return "Parfüm bulunamadı veya öneri oluşturulamadı."


In [64]:
print(df2['title'].unique())

['Vanilla Scent Fiorucci for women and men'
 'Shady Maiora Parfum for women and men'
 'Nerolia Vetiver Guerlain for women and men' ...
 'REMA Filigree & Shadow for women and men'
 'Wood Sage & Sea Salt Cologne Jo Malone London for women and men'
 'Passenger for Women S.T. Dupont for women']


In [66]:
# Örnek bir parfüm için öneriler al
sample_fragrance = "Vanilla Scent Fiorucci for women and men"  # Veya başka bir parfüm adı
recommendations = get_recommendations(sample_fragrance)

# Önerileri yazdır
print(f"\n'{sample_fragrance}' için Benzer Parfüm Önerileri:")
print(recommendations)


'Vanilla Scent Fiorucci for women and men' için Benzer Parfüm Önerileri:
                                         title  \
0           Mistura Aquaflor Firenze for women   
1  Vanilla Incense CJ Scents for women and men   
2          Vanille (Vanilla) Sephora for women   
3                        Scent #4 Zara for men   
4     Vanilla Vibes Birkholz for women and men   

                                 designer  similarity  rating  
0  aquaflor firenze perfumes and colognes    0.440961    4.00  
1         cj scents perfumes and colognes    0.339379    4.50  
2           sephora perfumes and colognes    0.306996    4.04  
3              zara perfumes and colognes    0.294706    3.75  
4          birkholz perfumes and colognes    0.290366    3.90  


### En popüler markaları bul

In [43]:
brand_counts = df2['designer'].value_counts().head(10)
print("\nEn Popüler 10 Parfüm Markası:")
print(brand_counts)


En Popüler 10 Parfüm Markası:
designer
avon perfumes and colognes                 48
zara perfumes and colognes                 32
bath & body works perfumes and colognes    24
the dua brand perfumes and colognes        20
natura perfumes and colognes               18
victoria's secret perfumes and colognes    18
rasasi perfumes and colognes               16
alkemia perfumes perfumes and colognes     15
dzintars perfumes and colognes             14
givenchy perfumes and colognes             12
Name: count, dtype: int64


In [45]:
# Nota ailelerini tanımlayalım
note_families = {
    'çiçeksi': [
        'Rose', 'Jasmine', 'Lavender', 'Tuberose', 'Orange Blossom', 'Ylang-Ylang', 'Narcissus',
        'African Orange Flower', 'Lotus', 'Carnation', 'Lily', 'Water Lily', 'Wisteria', 'Orchid',
        'Cyclamen', 'Cherry Blossom', 'Apple Blossom', 'Tunisian Neroli', 'Black Rose', 'Damask Rose',
        'Rose de Mai', 'Turkish Rose', 'Egyptian Jasmine', 'White Rose', 'Green Mandarin', 'White Flowers',
        'Rose Petals', 'Mirabelle', 'Acácia', 'White Mimosa', 'Vanilla Flower', 'Tiare Flower', 'Wild Lavender',
        'Jasmine Tea', 'Passion Flower', 'Wisteria Flower', 'French Rose', 'White Lily', 'Heather',
        'Rose Water', 'Night Blooming Jasmine', 'Red Rose', 'Orange Blossom Absolute', 'Blood Orange',
        'White Grapefruit', 'Moroccan Jasmine', 'Lily of the Valley', 'Lime (Linden) Blossom',
        'Petitgrain Paraguay', 'Neroli Essence', 'Camelia', 'Wild Jasmine', 'Black Locust', 'Damask Plum',
        'Star Jasmine', 'Black Currant Blossom', 'Marigold', 'Almond Blossom', 'Plumeria', 'Frangipani', 'Sweet Pea',
        'Wild Rose', 'Neroli', 'Geranium', 'Violet', 'Iris', 'Freesia', 'Gardenia', 'Peony', 'Magnolia', 'Lilac',
        'Hibiscus', 'Chamomile', 'Honeysuckle', 'Daffodil', 'Sakura', 'Poppy', 'Tulip', 'Hyacinth', 
        'Lotus Flower', 'Sunflower', 'Meadow Flowers', 'Floral Notes'
    ],
    'odunsu': [
        'Cedar', 'Sandalwood', 'Patchouli', 'Vetiver', 'Pine', 'Fir', 'Virginia Cedar', 'Woody Notes', 'Oakmoss',
        'Atlas Cedar', 'Guaiac Wood', 'Palo Santo', 'Teak Wood', 'Hinoki', 'White Sandalwood', 'Mahogany',
        'Cedarmoss', 'Virginian Cedar', 'Brazilian Rosewood', 'Palisander Rosewood', 'Ebony Wood', 'Birch',
        'Agarwood (Oud)', 'Cashmere Wood', 'Brazilian Sandalwood', 'Red Cedar', 'Siberian Pine',
        'Cedarwood', 'Oud', 'Oak', 'Driftwood', 'Cedar Wood'
    ],
    'baharatlı': [
        'Cinnamon', 'Black Pepper', 'Nutmeg', 'Ginger', 'Cardamom', 'Saffron', 'Pink Pepper', 'Black Mustard',
        'Allspice', 'Bay Leaf', 'Black Mustard', 'White Pepper', 'Cinammon', 'Caraway', 'Sichuan Pepper',
        'Ceylon Cinnamon', 'Cumin', 'Madagascar Cloves', 'Caraway', 'Anise', 'Clove', 'Pepper', 'Spicy Notes',
        'Aromatic Spices', 'Thyme', 'Marjoram', 'Oregano', 'Spices', 'Star Anise', 'Fennel', 'Juniper Berries',
        'Paprika', 'Turmeric', 'Cloves', 'Coriander Seed'
    ],
    'meyveli': [
        'Pineapple', 'Citrus', 'Grapefruit', 'Strawberry', 'Raspberry', 'Pear', 'Bergamot', 'Mandarin Orange',
        'Black Currant', 'Lemon', 'Orange', 'Peach', 'Apple', 'Plum', 'Watermelon', 'White Peach', 'Blueberry',
        'Red Berries', 'Red Currant', 'Kiwi', 'Sour Cherry', 'Green Apple', 'Red Fruits', 'Red Apple', 'Clementine',
        'Tayberry', 'Kumquat', 'Green Mango', 'Chinese Osmanthus', 'Pear Blossom', 'Gooseberry', 'Passionfruit',
        'Plum', 'Tropical Fruits', 'Exotic Fruits', 'Guava', 'Mango', 'Indian Tuberose', 'Coconut Milk',
        'Coconut Water', 'Persimmon', 'Tiare Flower', 'Black Currant Leaf', 'Mirabelle', 'Bilberry',
        'Goji Berries', 'Lychee', 'Cranberry', 'Wild Berries', 'Lychee', 'Quince', 'Carambola (Star Fruit)',
        'Cherry', 'Pomegranate', 'Apricot', 'Nectarine', 'Fig', 'Date', 'Papaya', 'Pineapple', 'Banana',
        'Tangerine', 'Blackberry', 'Rhubarb', 'Passion Fruit', 'Dragon Fruit', 'Mulberry'
    ],
    'amberli': [
        'Amber', 'Vanilla', 'Tonka Bean', 'Musk', 'Benzoin', 'Myrrh', 'Ambrette (Musk Mallow)', 'Bourbon Vanilla',
        'Madagascar Vanilla', 'Vanilla Absolute', 'Ambergris', 'Opoponax', 'Castoreum', 'Vanilla Sugar',
        'White Amber', 'Ambreine', 'Resins', 'Patchouli', 'Vanilla Pod', 'Tahitian Vanilla', 'Mexican Vanilla',
        'Civet', 'Golden Amber', 'Black Amber', 'Amber Wood'
    ],
    'yeşil': [
        'Green Tea', 'Basil', 'Lemongrass', 'Galbanum', 'Cypress', 'Grass', 'Bamboo', 'Green Notes', 'Green Leaves',
        'Green Grass', 'Tagetes', 'Wild Grass', 'Hay', 'Rosebay Willowherb', 'Tomato Leaf', 'Lime Leaf',
        'Green Pear', 'Green Mandarin', 'Petitgrain', 'Green Sap', 'Bamboo Leaf', 'Algae', 'Watercress',
        'Green Accord', 'Mint', 'Green Notes', 'Clover', 'Ivy', 'Fig Leaf', 'Cucumber', 'Green Apple', 'Fern',
        'Violet Leaf', 'Spearmint', 'Peppermint', 'Eucalyptus', 'Artemisia', 'Absinthe', 'Vetiver', 'Oakmoss'
    ],
    'kuruyemiş': [
        'Almond', 'Pistachio', 'Hazelnut', 'Sweet Almond', 'Bitter Almond', 'Almond Milk', 'Chestnut',
        'Walnut', 'Cashew', 'Macadamia', 'Peanut', 'Pecan', 'Coconut', 'Pine Nut'
    ],
    'sitrus': [
        'Lemon', 'Orange', 'Lime', 'Tangerine', 'Amalfi Lemon', 'Grapefruit', 'Sicilian Lemon', 'Blood Orange',
        'Pomelo', 'Citron', 'Sicilian Citrus', 'Kumquat', 'Italian Lemon', 'Lemon Blossom', 'Sicilian Mandarin',
        'Italian Orange', 'Citruses', 'Lemon Verbena', 'Yuzu', 'Lemon Zest', 'Lemon Balm', 'Bergamot', 'Lime Blossom',
        'Mandarin Blossom', 'Lime Leaf', 'Citrus Leaf', 'Orange Peel', 'Lemon Leaf', 'Orange Tree', 'Citrus',
        'Mandarin Orange', 'Neroli', 'Bergamot'
    ],
    'deniz_su': [
        'Sea Notes', 'Water Notes', 'Sea water', 'Watery Notes', 'Algae', 'Salt', 'Seaweed', 'Sea Salt',
        'Marine notes', 'Aquatic Notes', 'Ocean', 'Beach', 'Coastal Air', 'Mineral Water', 'Fresh Water'
    ],
    'şekerli_tatlı': [
        'Whipped Cream', 'Licorice', 'Cacao', 'Mulled Wine', 'Beeswax', 'Candy Apple', 'Cotton Candy',
        'Brown Sugar', 'Toffee', 'Caramel', 'Sorbet', 'Spun Sugar', 'Praline', 'Cupcake', 'Rice Pudding',
        'Marshmallow', 'Vanilla Sugar', 'Carambola (Star Fruit)', 'Dark Chocolate', 'Milk', 'Butter',
        'Honey', 'Maple Syrup', 'Condensed Milk', 'Milk Chocolate', 'Sugar', 'Marshmallow', 'Turkish Delight',
        'Cream', 'White Chocolate', 'Tiramisu', 'Crème Brûlée', 'Nougat', 'Macaroon', 'Pastry', 'Marzipan'
    ],
    'otlu_aromatik': [
        'Clary Sage', 'Chamomile', 'Verbena', 'Artemisia', 'Elemi', 'Rhubarb', 'Sage', 'Rosemary', 'Labdanum',
        'Thyme', 'Cumin', 'Oregano', 'Green Pepper', 'Parsley', 'Marjoram', 'Coriander', 'Hyssop', 'Marigold',
        'Basil', 'Peppermint', 'Sage', 'Mint', 'Tarragon', 'Dill', 'Cilantro', 'Fennel', 'Angelica', 'Anise',
        'Lavender', 'Bay Leaf', 'Laurel', 'Juniper', 'Basil'
    ],
    'içki': [
        'Whiskey', 'Rum', 'Gin', 'Champagne', 'Cognac', 'Liquor', 'Vodka', 'Blue Margarita', 'Brandy',
        'Limoncello', 'Amaretto', 'Wine Lees', 'Beer', 'Sake', 'Bourbon', 'Scotch', 'Red Wine', 'White Wine',
        'Port Wine', 'Tequila', 'Absinthe', 'Calvados', 'Vermouth'
    ],
    'reçineli_balsamlı': [
        'Peru Balsam', 'Benzoin', 'Myrrh', 'Tolu Balsam', 'Copaiba balm', 'Elemi resin', 'Balsam Fir',
        'Somalian Opoponax', 'Gurjan Balsam', 'Frankincense', 'Opium', 'Pine Tar', 'Spruce', 'Ambergris',
        'Resin', 'Styrax', 'Olibanum', 'Labdanum', 'Dragon\'s Blood', 'Canadian Balsam', 'Mastic',
        'Siam Benzoin', 'Poplar Bud', 'Balsam', 'Resins'
    ],
    'deri': [
        'Leather', 'Black Leather', 'Russian Leather', 'Suede', 'Suede Leather', 'Skin', 'Nubuck',
        'Vintage Leather', 'White Leather', 'Tanned Leather'
    ],
    'tütsü': [
        'Incense', 'Olibanum', 'Kyara Incense', 'Frankincense', 'Myrrh', 'Oud Incense', 'Church Incense',
        'Nag Champa', 'Japanese Incense', 'Arabian Incense', 'Tibetan Incense'
    ],
    'ağaç': [
        'Bamboo', 'Palm Tree', 'Brazilian Rosewood', 'Palisander Rosewood', 'Ebony Wood', 'Birch', 'Hinoki',
        'Oak', 'Wood', 'Sandalwood', 'Oud Wood', 'Palo Santo', 'Driftwood', 'Teak', 'Maple', 'Cypress',
        'Juniper', 'Cedarwood', 'Pine'
    ],
    'sentetik': [
        'Aldehydes', 'Iso E Super', 'Ozonic notes', 'Concrete', 'Cashmeran', 'Metallic notes', 'Mineral notes',
        'Powdery Notes', 'Animal notes', 'Ambroxan', 'Calone', 'Ambrette', 'Synthetic Musk', 'Hedione', 
        'Javanol', 'Galaxolide', 'Amyl Salicylate', 'Norlimbanol', 'Dihydromyrcenol'
    ],
    'kahve_çay': [
        'Coffee', 'Coffee Bean', 'Espresso', 'Black Tea', 'Green Tea', 'Earl Grey Tea', 'White Tea', 'Matcha',
        'Oolong Tea', 'Chai', 'Mate', 'Roasted Coffee'
    ],
    'tütün': [
        'Tobacco', 'Tobacco Leaf', 'Pipe Tobacco', 'Blonde Tobacco', 'Cigar', 'Hookah Tobacco', 'Cigarette',
        'Virginia Tobacco', 'Turkish Tobacco', 'Tobacco Flower'
    ]
}

In [47]:
# Nota sütunundaki string listeleri gerçek listelere dönüştürelim
def extract_notes(notes_str):
    if pd.isna(notes_str):
        return []
    try:
        return ast.literal_eval(notes_str) if isinstance(notes_str, str) else [notes_str]
    except:
        return []

# Notaları gerçek liste haline dönüştürme
df2['notes_list'] = df2['notes'].apply(extract_notes)

# Nota ailelerine göre sınıflandırma yapalım
def categorize_note(note):
    for family, notes in note_families.items():
        if note in notes:
            return family
    return "Diğer"

# Notaları nota ailelerine göre sınıflandırma yapalım
df2['note_family'] = df2['notes_list'].apply(lambda x: [categorize_note(note) for note in x])

# "Diğer" kategorisindeki önemli notaları yeni kategorilere ekleyelim
note_families['meyveli'] += ['Pear', 'Green Apple', 'Mirabelle']
note_families['içki'] += ['Cognac', 'Wine']
note_families['reçineli_balsamlı'] += ['Balsam', 'Resins']
note_families['yeşil'] += ['Vetiver', 'Oakmoss']

In [48]:
# Gelişmiş nota analizi fonksiyonu
def advanced_note_analysis(df):
    # Nota ailelerinin yüzdelik dağılımı
    family_dist = df['note_family'].explode().value_counts(normalize=True).mul(100)
    
    # Parfüm başına düşen ortalama nota ailesi sayısı
    avg_families_per_perfume = df['note_family'].apply(len).mean()
    
    # En popüler kombinasyonlar
    combo_counts = Counter()
    for fam in df['note_family']:
        for r in range(2, 4):
            combo_counts.update(combinations(sorted(fam), r))
    
    # Rating ilişkisi
    family_ratings = {}
    for family in note_families:
        mask = df['note_family'].apply(lambda x: family in x)
        family_ratings[family] = df[mask]['rating'].mean()
    
    return family_dist, avg_families_per_perfume, combo_counts, family_ratings

# Gelişmiş analizi çalıştır
family_dist, avg_count, combos, ratings = advanced_note_analysis(df2)

In [49]:
print("📊 Nota Aile Dağılımı (%):")
print(family_dist.round(2).to_string())
print(f"\n🌺 Parfüm Başına Ortalama Nota Ailesi Sayısı: {avg_count:.1f}")

print("\n🔥 En Popüler 10 Kombinasyon:")
for combo, count in combos.most_common(10):
    print(f"{' + '.join(combo)}: {count}")

print("\n⭐ Nota Ailelerine Göre Ortalama Ratingler:")
for fam, score in sorted(ratings.items(), key=lambda x: x[1], reverse=True):
    print(f"{fam:<15} {score:.2f}")

# Marka analizi fonksiyonu
def brand_analysis(df):
    brand_stats = {}
    for brand in df['designer'].value_counts().head(10).index:
        brand_df = df[df['designer'] == brand]
        avg_rating = brand_df['rating'].mean()
        top_families = brand_df['note_family'].explode().value_counts().head(3)
        brand_stats[brand] = {'avg_rating': avg_rating, 'top_families': top_families}
    return brand_stats

print("\n🏆 En İyi 10 Marka Analizi:")
for brand, stats in brand_analysis(df2).items():
    print(f"\n{brand}")
    print(f"  Ortalama Rating: {stats['avg_rating']:.2f}")
    print("  Top 3 Nota Ailesi:")
    for fam, count in stats['top_families'].items():
        print(f"  - {fam}: {count} kez")

📊 Nota Aile Dağılımı (%):
note_family
çiçeksi              19.58
amberli              14.82
meyveli              14.30
odunsu               14.19
Diğer                13.78
baharatlı             7.31
yeşil                 3.48
otlu_aromatik         2.34
şekerli_tatlı         1.91
sitrus                1.72
sentetik              1.11
deri                  1.06
reçineli_balsamlı     1.05
kuruyemiş             0.77
deniz_su              0.66
tütsü                 0.63
tütün                 0.47
içki                  0.40
kahve_çay             0.40
ağaç                  0.01

🌺 Parfüm Başına Ortalama Nota Ailesi Sayısı: 8.1

🔥 En Popüler 10 Kombinasyon:
amberli + meyveli + çiçeksi: 10167
amberli + odunsu + çiçeksi: 8713
meyveli + odunsu + çiçeksi: 8555
Diğer + amberli + çiçeksi: 7497
Diğer + meyveli + çiçeksi: 7399
amberli + çiçeksi + çiçeksi: 7019
Diğer + odunsu + çiçeksi: 6834
meyveli + çiçeksi + çiçeksi: 6561
amberli + meyveli + odunsu: 6231
amberli + çiçeksi: 5837

⭐ Nota Ailelerine Gö

## Analiz Sonuçları

### En Popüler 10 Parfüm
1. **Vanilla Scent** - Fiorucci for women and men - Rating: 5.0
2. **Satori Fusho Organic Oud** - Ensar Oud for women and men - Rating: 5.0
3. **Eau de Parfum Baies Exquises** - Durance en Provence for women and men - Rating: 5.0
4. **Amuro 7** - Dzintars for women - Rating: 5.0
5. **Горная фиалка (Mountain violet)** - Flora for women - Rating: 5.0
6. **Sky Blue** - Novae Plus for women - Rating: 5.0
7. **Orris TEAsense** - Mobius Fragrances for women and men - Rating: 5.0
8. **Legend** - FA'RA for men - Rating: 5.0
9. **Queen In The War** - Prince War for women - Rating: 5.0
10. **Un Baiser sur le Pont Neuf** - Condensé Paris for women and men - Rating: 5.0

### En Popüler Notalar
- **Musk**: 942 kez kullanılmış
- **Amber**: 694 kez kullanılmış
- **Jasmine**: 660 kez kullanılmış
- **Bergamot**: 657 kez kullanılmış
- **Sandalwood**: 646 kez kullanılmış

### Nota Aile Dağılımı (%)
- **Çiçeksi**: 19.58%
- **Amberli**: 14.82%
- **Meyveli**: 14.30%
- **Odunsu**: 14.19%
- **Diğer**: 13.78%

### Parfüm Başına Ortalama Nota Ailesi Sayısı
- Ortalama: **8.1**

### En Popüler 10 Kombinasyon
- **amberli + meyveli + çiçeksi**: 10167
- **amberli + odunsu + çiçeksi**: 8713
- **meyveli + odunsu + çiçeksi**: 8555

### Marka Analizi
- **Avon**: Ortalama Rating: 3.91, Top 3 Nota Ailesi: Diğer, Meyveli, Çiçeksi
- **Zara**: Ortalama Rating: 4.07, Top 3 Nota Ailesi: Meyveli, Çiçeksi, Odunsu
- **Bath & Body Works**: Ortalama Rating: 4.16, Top 3 Nota Ailesi: Meyveli, Çiçeksi, Diğer

Bu analiz, parfüm endüstrisindeki trendleri ve kullanıcı tercihlerini anlamak için önemli bilgiler sunmaktadır.